In [257]:
import pandas as pd
import json

def my_code():
# 1930 a 2010
    foot = pd.read_csv("data/matches_19302010.csv")
    foot["id_match"] = range(1, len(foot) + 1)
    round_dict = {
        "PRELIMINARY": "PRELIMINARY",
        "1/2_FINAL": "semifinal",
        "1/4_FINAL": "quarter",
        "1/8_FINAL": "round_16",
        "_FINAL": "final",
        "QUARTERFINAL_STAGE": "quarter",
        "GROUP_STAGE": "groups",
        "FINAL_ROUND": "groups",
        "PLACES_3&4": "final_loser",
        "FIRST": "groups",
        "SEMIFINAL_STAGE": "round_16"
    }
    foot["round"] = foot["round"].replace(round_dict)
    foot = foot.rename({'team1': 'home_team', 'team2': 'away_team', 'year': 'date', 'venue': 'city'}, axis='columns')
    foot = foot[["id_match", "home_team", "away_team", "score", "date", "city", "round", "edition"]]
    foot[["home_result", "away_result"]] = foot["score"].str.extract(r"(\d+)-(\d+)")
    foot = foot[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    data = foot.drop(foot[foot["edition"] == "2014-BRAZIL"].index)
    
# 2014
    foot = pd.read_csv("data/WorldCupMatches2014.csv", sep=";")
    foot["edition"] = "2014-BRAZIL"
    foot["id_match"] = range(len(data) + 1, len(data) + len(foot) + 1)
    foot = foot.rename({'id_match': "id_match", 'Home Team Name': "home_team", 'Away Team Name': "away_team", 
    'Home Team Goals':  "home_result", 'Away Team Goals': "away_result", 'Year': "date", 
     'Stage': "round", 'edition': "edition"}, axis='columns')
    foot = foot[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    data = pd.concat([data, foot], ignore_index=True)
    
# 2018
    with open("data/data_2018.json", "r") as f:
        foot = json.load(f)

    teams = foot["teams"]
    stadiums = foot["stadiums"]

    temp = foot["knockout"]["round_16"]["matches"]
    temp = pd.DataFrame(temp)
    temp["round"] = "round_16"
    matches = temp

    temp = foot["knockout"]["round_8"]["matches"]
    temp = pd.DataFrame(temp)
    temp["round"] = "quarter"
    matches = matches.append(temp)

    temp = foot["knockout"]["round_4"]["matches"]
    temp = pd.DataFrame(temp)
    temp["round"] = "semifinal"
    matches = matches.append(temp)

    temp = foot["knockout"]["round_2_loser"]["matches"]
    temp = pd.DataFrame(temp)
    temp["round"] = "final_loser"
    matches = matches.append(temp)

    temp = foot["knockout"]["round_2"]["matches"]
    temp = pd.DataFrame(temp)
    temp["round"] = "final"
    matches = matches.append(temp)

    for i in ['a','b','c','d','e','f','g','h']:
        temp = foot["groups"][i]["matches"]
        temp = pd.DataFrame(temp)
        temp["round"] = "groups"
        matches = matches.append(temp)
        
    matches["id_match"] = range(len(data) + 1, len(data) + len(matches) + 1)
    matches["edition"] = "2018-RUSSIA"
    
    matches = matches[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    teams = pd.DataFrame(teams)
    matches = pd.merge(matches, teams, left_on="home_team", right_on="id", how="left")
    matches["home_team"] = matches["name"]
    matches = matches[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    matches = pd.merge(matches, teams, left_on="away_team", right_on="id", how="left")
    matches["away_team"] = matches["name"]
    matches = matches[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    matches
    data = pd.concat([data, matches], ignore_index=True)
    
# 2022
    foot = pd.read_csv("data/WorldCupMatches2022.csv", sep=",")
    foot["edition"] = "2022-QATAR"
    foot["id_match"] = range(len(data) + 1, len(data) + len(foot) + 1)
    foot = foot.rename({'id_match': "id_match", 'team1': "home_team", 'team2': "away_team", 
    'number of goals team1':  "home_result", 'number of goals team2': "away_result", 'date': "date", 
     'category': "round", 'edition': "edition"}, axis='columns')
    foot = foot[["id_match", "home_team", "away_team", "home_result", "away_result", "date", "round", "edition"]]
    data = pd.concat([data, foot], ignore_index=True)
    
    pass

In [258]:
data.to_csv('data/fifa_final.csv', index=False, sep=';')

In [259]:
import timeit

execution_time = timeit.timeit(my_code, number=50)/50
print(f'Execution time: {execution_time} seconds')

Execution time: 0.058867691999985254 seconds


In [254]:
data

,id_match,home_team,away_team,home_result,away_result,date,round,edition
0,1,France,Mexico (México),4,1,1930,groups,1930-URUGUAY
1,2,USA,Belgium (België),3,0,1930,groups,1930-URUGUAY
2,3,Yugoslavia (Југославија),Brazil (Brasil),2,1,1930,groups,1930-URUGUAY
3,4,Romania (România),Peru (Perú),3,1,1930,groups,1930-URUGUAY
4,5,Argentina,France,1,0,1930,groups,1930-URUGUAY
...,...,...,...,...,...,...,...,...
6682,6619,ENGLAND,FRANCE,1,2,10 DEC 2022,Quarter-final,2022-QATAR
6683,6620,ARGENTINA,CROATIA,3,0,13 DEC 2022,Semi-final,2022-QATAR
6684,6621,FRANCE,MOROCCO,2,0,14 DEC 2022,Semi-final,2022-QATAR
6685,6622,CROATIA,MOROCCO,2,1,17 DEC 2022,Play-off for third place,2022-QATAR


In [243]:
data.sample(20).to_csv('data/fifa_sample.csv', index=False, sep=';')